#### DNN 기반 회귀 모델 구현 + 학습 진행 모니터링
- 데이터셋 : iris.csv
- Feature : 3개 Sepal_Length, Sepal_Width, Petal_Length
- Target : 1개 Petal_Width
- 학습-방법 : 지도학습 > 회귀
- 알고리즘 : 인공신경망(ANN) => MLP, DNN : 은닉층이 많은 구성
- 프레임워크 : Pytorch
* * *
- 모니터링
    * 기준 : 검증데이터셋의 loss 또는 score
    * 평가 : 학습데이터셋의 loss 또는 score와 비교해서 학습 중단여부 결정
    * 선택 : 현재까지 진행된 모델의 파라미터(가중치, 절편) 저장 여부 또는 모델 전체 저장
    


[1] 모듈 로딩 및 데이터 준비 <hr>

In [22]:
# 모델 관련 모듈 로딩
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from torchmetrics.regression import R2Score, MeanSquaredError
from torchinfo import summary

# Data 관련 모듈 로딩
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import *
from sklearn.model_selection import train_test_split

In [23]:
# 활용 패키지 버전 체크 => 사용자 정의 함수로 구현하기
print(f'Pytorch V. {torch.__version__}')
print(f'Pandas V. {torch.__version__}')

Pytorch V. 2.4.1+cpu
Pandas V. 2.4.1+cpu


In [24]:
# 데이터 로딩
DATA_FILE='../data/iris.csv'

# CSV => DataFrame
irisDF=pd.read_csv(DATA_FILE, usecols=[0,1,2,3])

# 데이터 확인
irisDF.head(1)

,sepal.length,sepal.width,petal.length,petal.width
0,5.1,3.5,1.4,0.2


[2] 모델 클래스 설계 및 정의<hr>
- 클래스목적 : iris 데이터를 학습 및 추론 목적 
- 클래스이름 : IrisRegModel
- 부모클래스 : nn.Module
- 매개변수 : 층별 입출력 개수 고정하기때문에 필요 없음
- 속성필드 : 
- 기능역할 : __init__() : 모델 구조, forward() : 순방향 학습 <= 오버라이딩
- 클래스구조
    * 입력층 : 입력 3개(피처)   출력 10개(퍼셉트론/뉴런 10개 존재)
    * 은닉층 : 입력 10개        출력 30개(퍼셉트론/뉴런 30개 존재)
    * 출력층 : 입력 30개        출력 1개(너비값)

- 활성화함수
    * 클래스형태 => nn.MESLoss, nn.ReLU => __init__() 메서드
    * 함수형태 => torch.nn.fuctional 아래에 => forward() 메서드

In [25]:
class IrisRegModel(nn.Module):

    # 모델 구조 구성 및 인스턴스 생성 메서드
    def __init__(self):
        super().__init__()

        self.in_layer=nn.Linear(3,10)
        self.hd_layer=nn.Linear(10,30)
        self.out_layer=nn.Linear(30,1)

    # 순방향 학습 진행 메서드
    def forward(self, x):
        y=F.relu(self.in_layer(x))
        y=F.relu(self.hd_layer(y))
        return self.out_layer(y)      # 회귀라서 바로 리턴

In [26]:
# 모델 인스턴스 생성
model=IrisRegModel()

print(model)

IrisRegModel(
  (in_layer): Linear(in_features=3, out_features=10, bias=True)
  (hd_layer): Linear(in_features=10, out_features=30, bias=True)
  (out_layer): Linear(in_features=30, out_features=1, bias=True)
)


In [27]:
# 모델 사용 메모리 정보 확인
summary(model, input_size=(10,3))

Layer (type:depth-idx)                   Output Shape              Param #
IrisRegModel                             [10, 1]                   --
├─Linear: 1-1                            [10, 10]                  40
├─Linear: 1-2                            [10, 30]                  330
├─Linear: 1-3                            [10, 1]                   31
Total params: 401
Trainable params: 401
Non-trainable params: 0
Total mult-adds (M): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.01

[3] 데이터셋 클래스 설계 및 정의<hr>
- 데이터셋 : iris.csv
- 피쳐개수 : 3개
- 타겟개수 : 1개
- 클래스이름 : IrisDataset
- 부모클래스 : utils.data.Dataset
- 속성필드 : featureDF, targetDF, n_rows, n_features
- 필수메서드
    * __init__(self) : 데이터셋 저장 및 전처리, 개발자가 필요한 속성 설정
    * __len__(self) : 데이터의 개수 반환
    * __getitem__(self, index) : 특정 인덱스의 피쳐와 타겟 반환

In [28]:
class IrisDataset(Dataset):

    def __init__(self, featureDF, targetDF):
        self.featureDF=featureDF
        self.targetDF=targetDF
        self.n_rows=featureDF.shape[0]
        self.n_features=featureDF.shape[1]

    def __len__(self):
        return self.n_rows
    
    def __getitem__(self, index):
        # 넘파이를 텐서로
        featureTS=torch.FloatTensor(self.featureDF.iloc[index].values)
        targetTS=torch.FloatTensor(self.targetDF.iloc[index].values)        
        # 피쳐와 타겟 반환
        return featureTS, targetTS

[3-1] 데이터셋 인스턴스 생성 <hr>

In [29]:
# 피쳐, 타겟 추출
featureDF, targetDF=irisDF[irisDF.columns[:-1]], irisDF[irisDF.columns[-1:]]
print(f'featureDF => {featureDF.shape}, targetDF => {targetDF.shape}')

# 데이터셋 인스턴스 생성
irisDS=IrisDataset(featureDF, targetDF)

featureDF => (150, 3), targetDF => (150, 1)


[4] 학습 준비
- 학습 횟수 : EPOCH <= 처음부터 끝까지 학습하는 단위
- 배치 크기 : BATCH_SIZE <= 한번에 학습할 데이터셋 양
- 위치 지정 : DEVICE <= 텐서 저장 및 실행 위치 (GPU/CPU)
- 학습률 : LR 가중치와 절편 업데이트 시 경사하강법으로 업데이트 간격 설정 0.001~0.1

In [30]:
# 학습 진행 관련 설정
EPOCH=1
BATCH_SIZE=10
BATCH_CNT=irisDF.shape[0]/BATCH_SIZE
DEVICE= 'cuda' if torch.cuda.is_available() else 'cpu'
LR=0.001

In [31]:
print(DEVICE)

cpu


- 인스턴스/객체 : 모델, 데이터셋, 최적화 + (손실함수 , 성능지표)

In [32]:
# 모델 인스턴스
model=IrisRegModel()

# 데이터셋 인스턴스
X_train, X_test, y_train, y_test=train_test_split(featureDF, targetDF, random_state=1)
X_train, X_val, y_train, y_val=train_test_split(X_train, y_train, random_state=1)
print(f'{X_train.shape} {X_test.shape} {X_val.shape}')
print(f'{y_train.shape} {y_test.shape} {y_val.shape}')

trainDS=IrisDataset(X_train, y_train)
valDS=IrisDataset(X_val, y_val)
testDS=IrisDataset(X_test, y_test)

# 데이터로드 인스턴스
trainDL=DataLoader(trainDS, batch_size=BATCH_SIZE)

(84, 3) (38, 3) (28, 3)
(84, 1) (38, 1) (28, 1)


In [33]:
# [테스트] 데이터로더
for feature, target in trainDL:
    print(feature.shape, target.shape)

torch.Size([10, 3]) torch.Size([10, 1])
torch.Size([10, 3]) torch.Size([10, 1])
torch.Size([10, 3]) torch.Size([10, 1])
torch.Size([10, 3]) torch.Size([10, 1])
torch.Size([10, 3]) torch.Size([10, 1])
torch.Size([10, 3]) torch.Size([10, 1])
torch.Size([10, 3]) torch.Size([10, 1])
torch.Size([10, 3]) torch.Size([10, 1])
torch.Size([4, 3]) torch.Size([4, 1])


In [34]:
# 최적화 인스턴스 => W, b 텐서 즉, model.parameters() 전달
optimizer=optim.Adam(model.parameters(), lr=LR)

# 손실함수 인스턴스 => 회귀, MSE, MAE, RMSE ...
regLoss=nn.MSELoss()

In [35]:
# 학습의 효과 확인 손실값과 성능평가값 저장 필요
LOSS_HISTORY, SCORE_HISTROY=[[],[]], [[],[]]

## 학습 모니터링/스케줄링 설정 
# => LOSS_HISTORY, SCORE_HISTROY 활용
# => 임계기준 : 10번
BREAK_CNT = 0
LIMIT_VALUE = 10


for epoch in range(EPOCH):
    # 학습 모드로 모델 설정
    model.train()

    # 배치크기 만큼 데이터 로딩해서 학습 진행
    loss_total, score_total=0, 0
    for featureTS, targetTS in trainDL:

        # 학습 진행
        pre_y=model(featureTS)

        # 손실 계산
        loss=regLoss(pre_y, targetTS)
        loss_total += loss.item()

        # 성능평가 계산
        score=R2Score()(pre_y, targetTS)
        score_total += score.item()

        # 최적화 진행
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # 에포크 당 검증기능
    # 모델 검증 모드 설정
    model.eval()
    with torch.no_grad():

        # 검증 데이터셋
        val_featureTS=torch.FloatTensor(valDS.featureDF.values)
        val_targetTS=torch.FloatTensor(valDS.targetDF.values)

        # 추론/평가
        pre_val=model(val_featureTS)

        # 손실
        loss_val=regLoss(pre_val, val_targetTS)

        # 성능평가
        score_val=R2Score()(pre_val, val_targetTS)
    
    # 에포크 당 손실값과 성능평가값 저장
    LOSS_HISTORY[0].append(loss_total/BATCH_CNT)
    SCORE_HISTROY[0].append(score_total/BATCH_CNT)

    LOSS_HISTORY[1].append(loss_val)
    SCORE_HISTROY[1].append(score_val)

    # 학습 진행 모니터링 / 스케줄링 - 검증 DS 기준 => Loss와 Score 중 둘중 한 개만 보면 됨.
    # Loss 기준
    if len(LOSS_HISTORY[0]) >= 2:
        if LOSS_HISTORY[1][-1] >= LOSS_HISTORY[1][-2]: BREAK_CNT += 1

    # Score 기준
    if len(SCORE_HISTROY[0]) >= 2:
        if SCORE_HISTROY[1][-1] <= SCORE_HISTROY[1][-2]: BREAK_CNT += 1

    # 학습 중단 여부 설정
    if BREAK_CNT > LIMIT_VALUE:
        print('성능 및 손실 개선이 없어서 학습 중단')

- 모델 저장 방법 <hr>
* 방법1 : 모델 파라미터만 저장
* 방법2 : 모델 설계 구조 및 파라미터까지 모두 저장

In [36]:
# 학습된 모델 파라미터 값 확인
model.state_dict()

OrderedDict([('in_layer.weight',
              tensor([[-0.2276,  0.5473,  0.2135],
                      [-0.1290, -0.1506,  0.1949],
                      [ 0.0176, -0.2590, -0.4353],
                      [-0.3275,  0.2157, -0.4238],
                      [-0.0518,  0.4957,  0.3478],
                      [-0.0082, -0.1485,  0.1133],
                      [-0.5492,  0.3839,  0.1815],
                      [ 0.4047, -0.1220,  0.5707],
                      [ 0.2097,  0.2760, -0.0055],
                      [ 0.2715,  0.0405,  0.1818]])),
             ('in_layer.bias',
              tensor([ 0.0688, -0.1285, -0.2274, -0.3039,  0.4169,  0.3235, -0.4621,  0.2518,
                      -0.0622, -0.2341])),
             ('hd_layer.weight',
              tensor([[ 0.1251, -0.1339,  0.2973,  0.0345, -0.3089,  0.0779, -0.0489, -0.1034,
                        0.3098, -0.0367],
                      [-0.0987,  0.0421, -0.0111, -0.0404, -0.0063,  0.1542, -0.0606,  0.3087,
                     

* [방법 1] 모델 파라미터 즉, 층별 가중치와 절편들

In [37]:
### models 폴더 아래 프로젝트 폴더 아래 모델 파일 저장
import os

# 저장 경로
SAVE_PATH = '../models/iris/'
# 저장 파일명
SAVE_FILE = 'model_train_wb.pth'

In [38]:
# 경로상 폴더 존재 여부 체크
if not os.path.exists(SAVE_PATH):
    os.makedirs(SAVE_PATH) # 폴더/폴더/... 하위 폴더 까지 생성

In [39]:
# 모델 저장
torch.save(model.state_dict(),SAVE_PATH+SAVE_FILE)

In [40]:
# 모델 즉,가중치와 절편 로딩
# [1] 가중치와 절편 객체로 로딩
# [2] 모델의 state_dict 속성에 저장

# 읽기
wbTS=torch.load(SAVE_PATH+SAVE_FILE,weights_only=True)
print(type(wbTS))

<class 'collections.OrderedDict'>


In [41]:
# 모델 인스턴스에 저장
model2=IrisRegModel()  # 층마다 W,b 초기화

model2.load_state_dict(wbTS)

<All keys matched successfully>

In [42]:
print(model2)

IrisRegModel(
  (in_layer): Linear(in_features=3, out_features=10, bias=True)
  (hd_layer): Linear(in_features=10, out_features=30, bias=True)
  (out_layer): Linear(in_features=30, out_features=1, bias=True)
)
